In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pandas as pd
import utils.functions as functions
from utils.functions import *
import importlib
import json
from utils.graph_repair import repair_network
import matplotlib.patches as patches

Please install pyparsing, pydot & graphviz if you want to draw graphs or lattice diagrams


In [2]:
cmap = get_colormap()
input_file = 'dataset/worms'

In [6]:
worm_dicts = read_input_mat_file(input_file, remove_trend=True, smooth_spikes=True, mean_zero=True)